# Lab 2 - Spark
This lab will demonstrate how to perform web server log analysis with Spark. Log data is a very large, common data source and contains a rich set of information.  It comes from many sources, such as web, file, and compute servers, application logs, user-generated content,  and can be used for monitoring servers, improving business and customer intelligence, building recommendation systems, fraud detection, and much more. This lab will show you how to use Spark on real-world text-based production logs and fully harness the power of that data.

### Apache Web Server Log file format
The log files that we use for this assignment are in the [Apache Common Log Format (CLF)](http://httpd.apache.org/docs/1.3/logs.html#common) format. The log file entries produced in CLF will look something like this:
`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`
 
Each part of this log entry is described below.
* **`127.0.0.1`:** this is the IP address (or host name, if available) of the client (remote host) which made the request to the server. 
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from remote machine) is not available.
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from local logon) is not available. 
* **`[01/Aug/1995:00:00:01 -0400]`:** the time that the server finished processing the request. The format is:
`[day/month/year:hour:minute:second timezone]`.
* **`"GET /images/launch-logo.gif HTTP/1.0"`:** this is the first line of the request string from the client. It consists of a three components: the request method (e.g., `GET`, `POST`, etc.), the endpoint, and the client protocol version. 
* **`200`:** this is the status code that the server sends back to the client. This information is very valuable, because it reveals whether the request resulted in a successful response (codes beginning in 2), a redirection (codes beginning in 3), an error caused by the client (codes beginning in 4), or an error in the server (codes beginning in 5). The full list of possible status codes can be found in the HTTP specification ([RFC 2616](https://www.ietf.org/rfc/rfc2616.txt) section 10).
* **`1839`:** the last entry indicates the size of the object returned to the client, not including the response headers. If no content was returned to the client, this value will be "-" (or sometimes 0).
 
Using the CLF as defined above, we create a regular expression pattern to extract the nine fields of the log line. The function returns a pair consisting of a Row object and 1. If the log line fails to match the regular expression, the function returns a pair consisting of the log line string and 0. A '-' value in the content size field is cleaned up by substituting it with 0. The function converts the log line's date string into a `Cal` object using the given `parseApacheTime` function. We, then, create the primary RDD and  we'll use in the rest of this assignment. We first load the text file and convert each line of the file into an element in an RDD. Next, we use `map(parseApacheLogLine)` to apply the parse function to each element and turn each line into a pair `Row` object. Finally, we cache the RDD in memory since we'll use it throughout this notebook. The log file is available at `data/apache/apache.log`.

In [26]:
import scala.util.matching
import org.apache.spark.rdd.RDD

case class Cal(year: Int, month: Int, day: Int, hour: Int, minute: Int, second: Int)                

case class Row(host: String, clientID: String, userID: String, dateTime: Cal, method: String, endpoint: String,
               protocol: String, responseCode: Int, contentSize: Long)                

val month_map = Map("Jan" -> 1, "Feb" -> 2, "Mar" -> 3, "Apr" -> 4, "May" -> 5, "Jun" -> 6, "Jul" -> 7, "Aug" -> 8,
                    "Sep" -> 9, "Oct" -> 10, "Nov" -> 11, "Dec" -> 12)

//------------------------------------------------
// A regular expression pattern to extract fields from the log line
val APACHE_ACCESS_LOG_PATTERN = """^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)""".r

//------------------------------------------------
def parseApacheTime(s: String): Cal = {
    return Cal(s.substring(7, 11).toInt, month_map(s.substring(3, 6)), s.substring(0, 2).toInt, 
               s.substring(12, 14).toInt, s.substring(15, 17).toInt, s.substring(18, 20).toInt)
}

//------------------------------------------------
def parseApacheLogLine(logline: String): (Either[Row, String], Int) = {
    val ret = APACHE_ACCESS_LOG_PATTERN.findAllIn(logline).matchData.toList
    if (ret.isEmpty)
        return (Right(logline), 0)

    val r = ret(0)
    val sizeField = r.group(9)

    var size: Long = 0
    if (sizeField != "-")
        size = sizeField.toLong

    return (Left(Row(r.group(1), r.group(2), r.group(3), parseApacheTime(r.group(4)), r.group(5), r.group(6),
                     r.group(7), r.group(8).toInt, size)), 1)
}

//------------------------------------------------
def parseLogs(): (RDD[(Either[Row, String], Int)], RDD[Row], RDD[String]) = {
    val fileName = "data/apache/apache.log"
    
    val parsedLogs = sc.textFile(fileName).map(parseApacheLogLine).cache()
    val accessLogs = parsedLogs.filter(x => x._2 == 1).map(x => x._1.left.get)
    val failedLogs = parsedLogs.filter(x => x._2 == 0).map(x => x._1.right.get)

    val failedLogsCount = failedLogs.count()
    
    if (failedLogsCount > 0) {
        println(s"Number of invalid logline: $failedLogs.count()")
        failedLogs.take(20).foreach(println)
    }
    
    println(s"Read $parsedLogs.count() lines, successfully parsed $accessLogs.count() lines, and failed to parse $failedLogs.count()")
    
    return (parsedLogs, accessLogs, failedLogs)
}

defined class Cal
defined class Row
month_map = Map(Nov -> 11, Jul -> 7, Mar -> 3, Jan -> 1, Oct -> 10, Dec -> 12, Feb -> 2, May -> 5, Apr -> 4, Aug -> 8, Sep -> 9, Jun -> 6)
APACHE_ACCESS_LOG_PATTERN = ^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)


parseApacheTime: (s: String)Cal
parseApacheLogLine: (logline: String)(Either[Row,String], Int)
parseLogs: ()(org.apache.spark.rdd.RDD[(Either[Row,String], Int)], org.apache.spark.rdd.RDD[Row], org.apache.spark.rdd.RDD[String])


^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)

In [2]:
val (parsedLogs, accessLogs, failedLogs) = parseLogs()

Read MapPartitionsRDD[2] at map at <console>:57.count() lines, successfully parsed MapPartitionsRDD[4] at map at <console>:58.count() lines, and failed to parse MapPartitionsRDD[6] at map at <console>:59.count()


parsedLogs = MapPartitionsRDD[2] at map at <console>:57
accessLogs = MapPartitionsRDD[4] at map at <console>:58
failedLogs = MapPartitionsRDD[6] at map at <console>:59


MapPartitionsRDD[6] at map at <console>:59

(4,1)
(16,1)
(8,1)
(12,1)
(20,1)
(19,1)
(15,1)
(11,1)
(3,1)
(7,1)
(22,1)
(14,1)
(6,1)
(18,1)
(10,1)
(13,1)
(21,1)
(1,1)
(17,1)
(9,1)
(5,1)
(8,CompactBuffer((8,nc3113ws144.nosc.mil), (8,mac114.cs.orst.edu), (8,140.109.247.32), (8,149.25.234.50), (8,dum37.interserve.com.hk), (8,slip9-174.beta.delphi.com), (8,plunders.demon.co.uk), (8,kwpoore.b30.ingr.com), (8,hall9.recitation.udel.edu), (8,mac0.cc.utah.edu), (8,dd08-024.compuserve.com), (8,139.169.59.106), (8,columbia.acc.brad.ac.uk), (8,ix-ftw-tx2-01.ix.netcom.com), (8,164.110.209.14), (8,ix-pa7-02.ix.netcom.com), (8,slip-42-6.ots.utexas.edu), (8,ibbeta26.ppp.usit.net), (8,ice.snowhill.com), (8,155.80.23.120), (8,godzilla.zeta.org.au), (8,proxy.halsp.hitachi.com), (8,cdr.vitinc.com), (8,libbabac-es.tamu.edu), (8,d2772a.bcasd.az.honeywell.com), (8,slip6.dsmnet.com), (8,markvogt.xnet.com), (8,ecch1-4-36.colorado.edu), (8,maciici.hmsc.orst.edu), (8,rusxppp23.rus.uni-stuttgart.de), (8,ppp26.localnet.com), (8,gate.funb.com), (8,slip2.nicoh.c

(9,CompactBuffer((9,philly53.voicenet.com), (9,gn109-has.uel.ac.uk), (9,outgate.ft.com), (9,ppp018-frhd.injersey.com), (9,ip227.b1.wsnet.com), (9,ip187.pay.primenet.com), (9,servpto06.uniandes.edu.co), (9,cust26.max1.philadelphia.pa.ms.uu.net), (9,slip74.wrq.com), (9,pc-048.isburg.ch), (9,pc-sebak.ee.umanitoba.ca), (9,mac1.bio.hartford.edu), (9,foster.qr.cas.uec.ac.jp), (9,163.205.16.102), (9,cola76.scsn.net), (9,tty5-22.swipnet.se), (9,libibm29.gatech.edu), (9,204.131.28.10), (9,watagashi.bekkoame.or.jp), (9,ns.okc.com), (9,cgl.bu.edu), (9,www.bconnex.net), (9,slip18.gulf.net), (9,synapse-08.mindport.net), (9,134.95.209.158), (9,ip027.lax.primenet.com), (9,152.72.80.240), (9,dialup-83.icon-stl.net), (9,hhenry.gsfc.nasa.gov), (9,pcasun8.gsfc.nasa.gov), (9,icon19.icon.hu), (9,jfch.vc.cvut.cz), (9,199.90.218.59), (9,163.205.16.110), (9,logic.lsil.com), (9,wepvr33.nottingham.ac.uk), (9,ix-rnwk1-12.ix.netcom.com), (9,drjo018a168.embratel.net.br), (9,ac10420.spjc.cc.fl.us), (9,penguin.atk.c

...

(14,CompactBuffer((14,isbe-node67.isbe.state.il.us), (14,concept.break.com.au), (14,as2511-10.sl010.cns.vt.edu), (14,dnet063.sat.texas.net), (14,dialup00016.cinet.itl.net), (14,n1121990.ksc.nasa.gov), (14,transducers4.ksc.nasa.gov), (14,206.12.170.214), (14,edr5.maf.nasa.gov), (14,198.60.24.86), (14,202.244.226.71), (14,130.110.205.20), (14,slip-3-15.shore.net), (14,136.152.93.216), (14,204.187.144.116), (14,cepheid.ugastro.berkeley.edu), (14,citlali.imt.mx), (14,204.29.16.1), (14,n1043362.ksc.nasa.gov), (14,137.82.38.175), (14,pm1-46.inx.net), (14,yellow.uah.edu), (14,halifax-ts1-43.nstn.ca), (14,h-dahlia.norfolk.infi.net), (14,165.112.172.37), (14,angel.esp.bellcore.com), (14,204.199.139.53), (14,abadon.stm.it), (14,xosgb3.dfrc.nasa.gov), (14,203.16.174.5), (14,153.20.24.68), (14,198.4.4.47), (14,pembroke.ftech.co.uk), (14,hdf166097.cad.ksc.nasa.gov), (14,158.152.158.41), (14,slip-3-1.shore.net), (14,l02.lrc.gasou.edu), (14,192.228.152.107), (14,hplvopen.lvld.hp.com), (14,tsa-25.ucsc

(19,CompactBuffer((19,interlock.turner.com), (19,h-darkhalf.norfolk.infi.net), (19,ianc.interlog.com), (19,ix-col1-21.ix.netcom.com), (19,gw1.magec.com), (19,cust7.max3.los-angeles.ca.ms.uu.net), (19,www-e1a.gnn.com), (19,csc.sctboces.org), (19,endora.vuse.vanderbilt.edu), (19,lmac84.jsc.nasa.gov), (19,n1127152.ksc.nasa.gov), (19,199.170.102.59), (19,ix-orm-ut1-08.ix.netcom.com), (19,spacelink.msfc.nasa.gov), (19,cardinal.excel.net), (19,buffnet1.buffnet.net), (19,128.227.114.39), (19,ad12-004.compuserve.com), (19,170.91.18.2), (19,portal-as2.labyrinth.net.au), (19,vance.cts.com), (19,203.67.36.2), (19,dialup48.av.ca.qnet.com), (19,128.236.124.87), (19,cochise.ppp.america.com), (19,p3.gallium.sentex.net), (19,dd19-009.compuserve.com), (19,nbc.ksu.ksu.edu), (19,mintaka.sdsu.edu), (19,p1dyn29.polaris.net), (19,plains.uwyo.edu), (19,143.206.180.14), (19,205.160.7.40), (19,ts5-2.slip.uwo.ca), (19,ix-cin3-08.ix.netcom.com), (19,ix-hay-ca1-19.ix.netcom.com), (19,rmcg.cts.com), (19,ix-hou8-04

(20,CompactBuffer((20,slip105.vianet.on.ca), (20,m027.fn.net), (20,slip56.crim.ca), (20,rdurham.mindspring.com), (20,enterprise.powerup.com.au), (20,line2.lausanne.ping.ch), (20,dialin06.kiss.de), (20,ts5-14.inforamp.net), (20,gateway.ps.net), (20,pm014-00.dialip.mich.net), (20,lilja.dial.hbg.lth.se), (20,dialup18-hugin.oden.se), (20,204.120.67.28), (20,nor134.vianet.on.ca), (20,nn1.tor.servtech.com), (20,dial196196.wbm.ca), (20,194.65.16.40), (20,www-a2.proxy.aol.com), (20,131.181.7.120), (20,asd01-19.dial.xs4all.nl), (20,ix-sj16-25.ix.netcom.com), (20,203.9.249.136), (20,dd14-014.compuserve.com), (20,slip24.niagara.com), (20,203.10.82.151), (20,ip-pdx4-03.teleport.com), (20,pld9.biddeford.com), (20,cust2.max1.seattle.wa.ms.uu.net), (20,future.dreamscape.com), (20,pm50.sonic.net), (20,ad066.du.pipex.com), (20,angelini.dialup.francenet.fr), (20,crc7-fddi.cris.com), (20,ix-dc6-22.ix.netcom.com), (20,mirac.unm.edu), (20,marvel.stsci.edu), (20,ix-jac1-24.ix.netcom.com), (20,151.99.142.206

(21,CompactBuffer((21,132.60.233.105), (21,140.121.155.12), (21,132.74.12.10), (21,macttfkry.desy.de), (21,stoneship.intellistor.com), (21,irpud1.raumplanung.uni-dortmund.de), (21,tayskilkfyys.tays.fi), (21,asyncb1.wincom.net), (21,mboo027.isc.rit.edu), (21,charlotte.anu.edu.au), (21,dal04-20.ppp.iadfw.net), (21,n103235.ksc.nasa.gov), (21,ix-bal1-02.ix.netcom.com), (21,cragateway.cra.com.au), (21,pm1-12.abc.se), (21,jaguar.colorado.edu), (21,dd16-045.compuserve.com), (21,sailor.lib.md.us), (21,139.169.52.224), (21,142.106.12.144), (21,groveend.netkonect.co.uk), (21,n164576.ksc.nasa.gov), (21,europa.gsf.fi), (21,146.232.34.37), (21,sol.racsa.co.cr), (21,ix-phi1-28.ix.netcom.com), (21,rulnas.leidenuniv.nl), (21,porta00.vision.net.au), (21,x217-205.cee.umn.edu), (21,moat.bnr.co.uk), (21,dial068.vanderbilt.edu), (21,van14394.direct.ca), (21,141.102.40.63), (21,lvarnell.jpl.nasa.gov), (21,129.38.233.50), (21,slip36-154.il.us.ibm.net), (21,modem10.mgl.ca), (21,148.232.11.20), (21,gyme.imv.aa

(17,CompactBuffer((17,slper1p24.ozemail.com.au), (17,atc.eag.unisysgsg.com), (17,fkirchman.gsfc.nasa.gov), (17,139.169.174.37), (17,l311s26.se12labs.bcit.bc.ca), (17,192.45.108.17), (17,chaos.es.co.nz), (17,slip1-32.acs.ohio-state.edu), (17,baloo.dc.luth.se), (17,163.205.154.12), (17,dd23-005.compuserve.com), (17,moran.pgd.hawaii.edu), (17,139.169.85.120), (17,163.206.137.11), (17,hermes.sce.carleton.ca), (17,ds22.ds.fht-mannheim.de), (17,neuron.bms.com), (17,n1032907.ksc.nasa.gov), (17,139.169.174.102), (17,uclg08.lng.uc.edu), (17,inetg1.arco.com), (17,euphrates.physics.iastate.edu), (17,202.242.141.5), (17,nrba1_onlink2.onlink.net), (17,max2-135.hiwaay.net), (17,next532-9.gsfc.nasa.gov), (17,homayra.gsfc.nasa.gov), (17,141.102.80.134), (17,149.156.233.22), (17,200.10.225.215), (17,atlas5.cern.ch), (17,d2637a.bcasd.az.honeywell.com), (17,128.159.122.38), (17,163.191.201.21), (17,berm.geology.uiuc.edu), (17,n1373377.ksc.nasa.gov), (17,cisco-slip116.acc.virginia.edu), (17,144.58.108.103

### Sample Analyses on the Web Server Log File
Let's compute some statistics about the sizes of content being returned by the web server. In particular, we'd like to know what are the average, minimum, and maximum content sizes. We can compute the statistics by applying a `map` to the `accessLogs` RDD. The given function to the `map` should extract the `contentSize` field from the RDD. The `map` produces a new RDD, called `contentSizes`, containing only the `contentSizes`. To compute the minimum and maximum statistics, we can use `min()` and `max()` functions on the new RDD. We can compute the average statistic by using the `reduce` function with a function that sums the two inputs, which represent two elements from the new RDD that are being reduced together. The result of the `reduce()` is the total content size from the log and it is to be divided by the number of requests as determined using the `count()` function on the new RDD. As the result of executing the following box, you should get the below result:
```
Content Size Avg: 17531, Min: 0, Max: 3421948
```

In [3]:
// TODO: Replace <FILL IN> with appropriate code
// Calculate statistics based on the content size.
val contentSizes = accessLogs.map(x => x.contentSize).cache()

val avg = contentSizes.reduce((x, y) => x+y) / contentSizes.count()

println("Content Size Avg: " + avg + ", Min: " + contentSizes.min() + ", Max: " + contentSizes.max())

Content Size Avg: 17531, Min: 0, Max: 3421948


contentSizes = MapPartitionsRDD[7] at map at <console>:31
avg = 17531


17531

Next, lets look at the "response codes" that appear in the log. As with the content size analysis, first we create a new RDD that contains the `responseCode` field from the `accessLogs` RDD. The difference here is that we will use a *pair tuple* instead of just the field itself (i.e., (response code, 1)). Using a pair tuple consisting of the response code and 1 will let us count the number of of records with a particular response code. Using the new RDD `responseCodes`, we perform a `reduceByKey` function that applys a given function to each element, pairwise with the same key. Then, we cache the resulting RDD and create a list by using the `take` function. Once you run the code below, you should receive the following results:
```
Found 7 response codes
Response Code Counts: (404,6185) (200,940847) (304,79824) (500,2) (501,17) (302,16244) (403,58)
```

In [4]:
// TODO: Replace <FILL IN> with appropriate code

// extract the response code for each record and make pair of (response code, 1)
val responseCodes = accessLogs.map(x => (x.responseCode, 1))

// count the number of records for each key
val responseCodesCount = responseCodes.reduceByKey((x, y) => x + y).cache()

// take the first 100 records
val responseCodesCountList = responseCodesCount.take(100)

println("Found " + responseCodesCountList.length + " response codes")
print("Response Code Counts: ")
responseCodesCountList.foreach(x => print(x + " "))

Found 7 response codes
Response Code Counts: (404,6185) (200,940847) (304,79824) (500,2) (501,17) (302,16244) (403,58) 

responseCodes = MapPartitionsRDD[8] at map at <console>:32
responseCodesCount = ShuffledRDD[9] at reduceByKey at <console>:35
responseCodesCountList = Array((404,6185), (200,940847), (304,79824), (500,2), (501,17), (302,16244), (403,58))


[(404,6185), (200,940847), (304,79824), (500,2), (501,17), (302,16244), (403,58)]

Let's look at "hosts" that have accessed the server multiple times (e.g., more than 10 times). First we create a new RDD to keep the `host` field from the `accessLogs` RDD using a pair tuple consisting of the host and 1 (i.e., (host, 1)), which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a given function to add the two values. We then filter the result based on the count of accesses by each host (the second element of each pair) being greater than 10. Next, we extract the host name by performing a `map` to return the first element of each pair. Finally, we extract 20 elements from the resulting RDD. The result should be as below:
```
Any 20 hosts that have accessed more then 10 times: 
ix-aug-ga1-13.ix.netcom.com 
n1043347.ksc.nasa.gov 
d02.as1.nisiq.net 
192.112.22.82 
anx3p4.trib.com 
198.215.127.2 
198.77.113.34 
crc182.cac.washington.edu 
telford-107.salford.ac.uk 
universe6.barint.on.ca 
gatekeeper.homecare.com 
157.208.11.7 
unknown.edsa.co.za 
onyx.southwind.net 
ppp-hck-2-12.ios.com 
ix-lv5-04.ix.netcom.com 
f-umbc7.umbc.edu 
cs006p09.nam.micron.net 
dd22-025.compuserve.com 
hak-lin-kim.utm.edu 
```

In [5]:
// TODO: Replace <FILL IN> with appropriate code

// extract the host field for each record and make pair of (host, 1)
val hosts = accessLogs.map(x => (x.host, 1))

// count the number of records for each key
val hostsCount = hosts.reduceByKey((x, y) => x+ y)

// keep the records with the count greater than 10
val hostMoreThan10 = hostsCount.filter( x => x._2 > 10 )

// take the first 100 records
val hostsPick20 = hostMoreThan10.map( x => x._1 ).take(20)

println("Any 20 hosts that have accessed more then 10 times: ")
hostsPick20.foreach(println)

Any 20 hosts that have accessed more then 10 times: 
ix-aug-ga1-13.ix.netcom.com
n1043347.ksc.nasa.gov
d02.as1.nisiq.net
192.112.22.82
anx3p4.trib.com
198.215.127.2
198.77.113.34
crc182.cac.washington.edu
telford-107.salford.ac.uk
universe6.barint.on.ca
gatekeeper.homecare.com
157.208.11.7
unknown.edsa.co.za
onyx.southwind.net
ppp-hck-2-12.ios.com
ix-lv5-04.ix.netcom.com
f-umbc7.umbc.edu
cs006p09.nam.micron.net
dd22-025.compuserve.com
hak-lin-kim.utm.edu


hosts = MapPartitionsRDD[10] at map at <console>:32
hostsCount = ShuffledRDD[11] at reduceByKey at <console>:35
hostMoreThan10 = MapPartitionsRDD[12] at filter at <console>:38
hostsPick20 = Array(ix-aug-ga1-13.ix.netcom.com, n1043347.ksc.nasa.gov, d02.as1.nisiq.net, 192.112.22.82, anx3p4.trib.com, 198.215.127.2, 198.77.113.34, crc182.cac.washington.edu, telford-107.salford.ac.uk, universe6.barint.on.ca, gatekeeper.homecare.com, 157.208.11.7, unknown.edsa.co.za, onyx.southwind.net, ppp-hck-2-12.ios.com, ix-lv5-04.ix.netcom.com, f-umbc7.umbc.edu, cs006p09.nam.micron.net, dd22-025.compuserve.com, hak-lin-kim.utm.edu)


[ix-aug-ga1-13.ix.netcom.com, n1043347.ksc.nasa.gov, d02.as1.nisiq.net, 192.112.22.82, anx3p4.trib.com, 198.215.127.2, 198.77.113.34, crc182.cac.washington.edu, telford-107.salford.ac.uk, universe6.barint.on.ca, gatekeeper.homecare.com, 157.208.11.7, unknown.edsa.co.za, onyx.southwind.net, ppp-hck-2-12.ios.com, ix-lv5-04.ix.netcom.com, f-umbc7.umbc.edu, cs006p09.nam.micron.net, dd22-025.compuserve.com, hak-lin-kim.utm.edu]

For the final example, we'll look at the top endpoints (URIs) in the log. To determine them, we first create a new RDD to extract the `endpoint` field from the `accessLogs` RDD using a pair tuple consisting of the endpoint and 1 (i.e., (endpoint, 1)), which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` to add the two values. We then extract the top 10 endpoints by performing a `takeOrdered` with a value of 10 and a function that multiplies the count (the second element of each pair) by -1 to create a sorted list with the top endpoints at the bottom. Here is the result:
```
Top ten endpoints: 
(/images/NASA-logosmall.gif,59737)
(/images/KSC-logosmall.gif,50452)
(/images/MOSAIC-logosmall.gif,43890)
(/images/USA-logosmall.gif,43664)
(/images/WORLD-logosmall.gif,43277)
(/images/ksclogo-medium.gif,41336)
(/ksc.html,28582)
(/history/apollo/images/apollo-logo1.gif,26778)
(/images/launch-logo.gif,24755)
(/,20292)
```

In [6]:
// TODO: Replace <FILL IN> with appropriate code

// extract the endpoint for each record and make pair of (endpoiint, 1)
val endpoints = accessLogs.map(x => (x.endpoint, 1))

// count the number of records for each key
val endpointCounts = endpoints.reduceByKey((x, y) => x+ y)

// extract the top 10 
val topEndpoints = endpointCounts.map(x => (x._2 *(-1), x._1)).takeOrdered(10).map(x => (x._2, x._1 * (-1)))

println("Top ten endpoints: ")
topEndpoints.foreach(println)

Top ten endpoints: 
(/images/NASA-logosmall.gif,59737)
(/images/KSC-logosmall.gif,50452)
(/images/MOSAIC-logosmall.gif,43890)
(/images/USA-logosmall.gif,43664)
(/images/WORLD-logosmall.gif,43277)
(/images/ksclogo-medium.gif,41336)
(/ksc.html,28582)
(/history/apollo/images/apollo-logo1.gif,26778)
(/images/launch-logo.gif,24755)
(/,20292)


endpoints = MapPartitionsRDD[14] at map at <console>:32
endpointCounts = ShuffledRDD[15] at reduceByKey at <console>:35
topEndpoints = Array((/images/NASA-logosmall.gif,59737), (/images/KSC-logosmall.gif,50452), (/images/MOSAIC-logosmall.gif,43890), (/images/USA-logosmall.gif,43664), (/images/WORLD-logosmall.gif,43277), (/images/ksclogo-medium.gif,41336), (/ksc.html,28582), (/history/apollo/images/apollo-logo1.gif,26778), (/images/launch-logo.gif,24755), (/,20292))


[(/images/NASA-logosmall.gif,59737), (/images/KSC-logosmall.gif,50452), (/images/MOSAIC-logosmall.gif,43890), (/images/USA-logosmall.gif,43664), (/images/WORLD-logosmall.gif,43277), (/images/ksclogo-medium.gif,41336), (/ksc.html,28582), (/history/apollo/images/apollo-logo1.gif,26778), (/images/launch-logo.gif,24755), (/,20292)]

### Analyzing Web Server Log File
What are the top ten endpoints which did not have return code 200? Create a sorted list containing top ten endpoints and the number of times that they were accessed with non-200 return code. Think about the steps that you need to perform to determine which endpoints did not have a 200 return code, how you will uniquely count those endpoints, and sort the list. You should receive the following result:
```
Top ten failed URLs: 
(/images/NASA-logosmall.gif,8761)
(/images/KSC-logosmall.gif,7236)
(/images/MOSAIC-logosmall.gif,5197)
(/images/USA-logosmall.gif,5157)
(/images/WORLD-logosmall.gif,5020)
(/images/ksclogo-medium.gif,4728)
(/history/apollo/images/apollo-logo1.gif,2907)
(/images/launch-logo.gif,2811)
(/,2199)
(/images/ksclogosmall.gif,1622)
```

In [18]:
// TODO: Replace <FILL IN> with appropriate code

// keep the logs with error code not 200
val not200 = accessLogs.filter(x => x.responseCode != 200)

// make a pair of (x, 1)
val endpointCountPairTuple = not200.map(x => (x.endpoint, 1))

// count the number of records for each key x
val endpointSum = endpointCountPairTuple.reduceByKey((x, y) => x + y)

// take the top 10
val topTenErrURLs = endpointSum.map(x => (x._2 *(-1), x._1)).takeOrdered(10).map(x => (x._2, x._1 * (-1)))

println("Top ten failed URLs: ")
topTenErrURLs.foreach(println)

Top ten failed URLs: 
(/images/NASA-logosmall.gif,8761)
(/images/KSC-logosmall.gif,7236)
(/images/MOSAIC-logosmall.gif,5197)
(/images/USA-logosmall.gif,5157)
(/images/WORLD-logosmall.gif,5020)
(/images/ksclogo-medium.gif,4728)
(/history/apollo/images/apollo-logo1.gif,2907)
(/images/launch-logo.gif,2811)
(/,2199)
(/images/ksclogosmall.gif,1622)


not200 = MapPartitionsRDD[18] at filter at <console>:32
endpointCountPairTuple = MapPartitionsRDD[19] at map at <console>:35
endpointSum = ShuffledRDD[20] at reduceByKey at <console>:38
topTenErrURLs = Array((/images/NASA-logosmall.gif,8761), (/images/KSC-logosmall.gif,7236), (/images/MOSAIC-logosmall.gif,5197), (/images/USA-logosmall.gif,5157), (/images/WORLD-logosmall.gif,5020), (/images/ksclogo-medium.gif,4728), (/history/apollo/images/apollo-logo1.gif,2907), (/images/launch-logo.gif,2811), (/,2199), (/images/ksclogosmall.gif,1622))


[(/images/NASA-logosmall.gif,8761), (/images/KSC-logosmall.gif,7236), (/images/MOSAIC-logosmall.gif,5197), (/images/USA-logosmall.gif,5157), (/images/WORLD-logosmall.gif,5020), (/images/ksclogo-medium.gif,4728), (/history/apollo/images/apollo-logo1.gif,2907), (/images/launch-logo.gif,2811), (/,2199), (/images/ksclogosmall.gif,1622)]

Let's count the number of unique hosts in the entire log. Think about the steps that you need to perform to count the number of different hosts in the log. The result should be as below:
```
Unique hosts: 54507

```

In [20]:
// TODO: Replace <FILL IN> with appropriate code

// extract the host field for each record
val hosts = accessLogs.map(x => x.host)

// keep the uniqe hosts
val uniqueHosts = hosts.distinct

// count them
val uniqueHostCount = uniqueHosts.count()

println("Unique hosts: " + uniqueHostCount)

Unique hosts: 54507


hosts = MapPartitionsRDD[23] at map at <console>:34
uniqueHosts = MapPartitionsRDD[26] at distinct at <console>:37
uniqueHostCount = 54507


54507

For an advanced exercise, let's determine the number of unique hosts in the entire log on a day-by-day basis. This computation will give us counts of the number of unique daily hosts. We'd like a list sorted by increasing day of the month, which includes the day of the month and the associated number of unique hosts for that day. Make sure you cache the resulting RDD `dailyHosts`, so that we can reuse it in the next exercise. Think about the steps that you need to perform to count the number of different hosts that make requests *each* day. Since the log only covers a single month, you can ignore the month. Here is the output you should receive:
```
Unique hosts per day: 
(1,2582)
(3,3222)
(4,4190)
(5,2502)
(6,2537)
(7,4106)
(8,4406)
(9,4317)
(10,4523)
(11,4346)
(12,2864)
(13,2650)
(14,4454)
(15,4214)
(16,4340)
(17,4385)
(18,4168)
(19,2550)
(20,2560)
(21,4134)
(22,4456)
```

In [85]:
// TODO: Replace <FILL IN> with appropriate code

// make pairs of (day, host)
val dayToHostPairTuple = accessLogs.map(x => (x.dateTime.day, x.host)).distinct

// group by day
val dayGroupedHosts = dayToHostPairTuple.groupBy( _._1 ).mapValues( _.map( _._2 ) )

// make pairs of (day, number of host in that day)
val dayHostCount = dayGroupedHosts.map(x => (x._1, x._2.size))

// sort by day
val dailyHosts = dayHostCount.sortByKey().cache()

// return the records as a list
val dailyHostsList = dailyHosts.take(30)

println("Unique hosts per day: ")
dailyHostsList.foreach(println)

Unique hosts per day: 
(1,2582)
(3,3222)
(4,4190)
(5,2502)
(6,2537)
(7,4106)
(8,4406)
(9,4317)
(10,4523)
(11,4346)
(12,2864)
(13,2650)
(14,4454)
(15,4214)
(16,4340)
(17,4385)
(18,4168)
(19,2550)
(20,2560)
(21,4134)
(22,4456)


dayToHostPairTuple = MapPartitionsRDD[71] at distinct at <console>:40
dayGroupedHosts = MapPartitionsRDD[74] at mapValues at <console>:43
dayHostCount = MapPartitionsRDD[75] at map at <console>:46
dailyHosts = ShuffledRDD[78] at sortByKey at <console>:49
dailyHostsList = Array((1,2582), (3,3222), (4,4190), (5,2502), (6,2537), (7,4106), (8,4406), (9,4317), (10,4523), (11,4346), (12,2864), (13,2650), (14,4454), (15,4214), (16,4340), (17,4385), (18,4168), (19,2550), (20,2560), (21,4134), (22,4456))


[(1,2582), (3,3222), (4,4190), (5,2502), (6,2537), (7,4106), (8,4406), (9,4317), (10,4523), (11,4346), (12,2864), (13,2650), (14,4454), (15,4214), (16,4340), (17,4385), (18,4168), (19,2550), (20,2560), (21,4134), (22,4456)]

Next, let's determine the average number of requests on a day-by-day basis. We'd like a list by increasing day of the month and the associated average number of requests per host for that day. Make sure you cache the resulting RDD `avgDailyReqPerHost` so that we can reuse it in the next exercise. To compute the average number of requests per host, get the total number of request across all hosts and divide that by the number of unique hosts. Since the log only covers a single month, you can skip checking for the month. Also to keep it simple, when calculating the approximate average use the integer value. The result should be as below:
```
Average number of daily requests per Hosts is: 
(1,13)
(3,12)
(4,14)
(5,12)
(6,12)
(7,13)
(8,13)
(9,14)
(10,13)
(11,14)
(12,13)
(13,13)
(14,13)
(15,13)
(16,13)
(17,13)
(18,13)
(19,12)
(20,12)
(21,13)
(22,12)
```

In [90]:
val dayAndHostTuple = accessLogs.map(x => (x.dateTime.day, x.host)).distinct
val groupedByDay = dayAndHostTuple.groupBy(_._1)


dayAndHostTuple = MapPartitionsRDD[86] at distinct at <console>:33
groupedByDay = ShuffledRDD[88] at groupBy at <console>:34


ShuffledRDD[88] at groupBy at <console>:34

In [91]:
val sortedByDay = groupedByDay.sortBy(_._1)


sortedByDay = MapPartitionsRDD[93] at sortBy at <console>:34


MapPartitionsRDD[93] at sortBy at <console>:34

In [93]:
for ( x <- sortedByDay.take(2)){
    println(x)
}

(1,CompactBuffer((1,206.24.43.11), (1,dsc.dfrc.nasa.gov), (1,linuser19.li.net), (1,130.160.246.88), (1,163.205.44.27), (1,192.72.189.1), (1,evitech.evitech.fi), (1,www-b4.proxy.aol.com), (1,bos95.pi.net), (1,128.217.62.24), (1,remote1.slip.tu-clausthal.de), (1,ad12-024.compuserve.com), (1,147.147.35.186), (1,164.124.101.6), (1,bzi.chem.ornl.gov), (1,moon.ee.umist.ac.uk), (1,n1127413.ksc.nasa.gov), (1,sasdns3.sas.usace.army.mil), (1,sao.gsfc.nasa.gov), (1,pc6.garden.bt.co.uk), (1,155.148.3.108), (1,proxy05.bell-atl.com), (1,146.105.30.3), (1,n1028279.ksc.nasa.gov), (1,patches.jpl.nasa.gov), (1,137.226.108.22), (1,128.158.36.4), (1,n1027537.ksc.nasa.gov), (1,smtp.inet.fi), (1,dezernsquadra650.larc.nasa.gov), (1,n1032154.ksc.nasa.gov), (1,199.72.6.62), (1,n1121722.ksc.nasa.gov), (1,ip219.vcv.primenet.com), (1,relay.ngic.osis.gov), (1,jrs4941.dukepower.com), (1,163.205.54.76), (1,163.205.12.7), (1,ecn1334067.gsfc.nasa.gov), (1,comserv-f-42.usc.edu), (1,gnpc115.uovs.ac.za), (1,grant.jhuapl.

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

// make pairs of (day, host)
val dayAndHostTuple = accessLogs.map(x => (x.dateTime.day, x.host)).distinct

// group by day
val groupedByDay = dayAndHostTuple.<FILL IN>

// sort by day
val sortedByDay = groupedByDay.<FILL IN>

// calculate the average request per day
val avgDailyReqPerHost = sortedByDay.map(<FILL IN>).cache()

// return the records as a list
val avgDailyReqPerHostList = avgDailyReqPerHost.take(30)

println("Average number of daily requests per Hosts is: ")
avgDailyReqPerHostList.foreach(println)

### Exploring 404 Response Codes
Let's count the 404 response codes. Create a RDD containing only log records with a 404 response code. Make sure you `cache()` the RDD `badRecords` as we will use it in the rest of this exercise. How many 404 records are in the log? Here is the result:
```
Found 6185 404 URLs.
```

In [95]:
// TODO: Replace <FILL IN> with appropriate code

val badRecords = accessLogs.filter(x => x.responseCode == 404)

println("Found " + badRecords.count() + " 404 URLs.")

Found 6185 404 URLs.


badRecords = MapPartitionsRDD[94] at filter at <console>:33


MapPartitionsRDD[94] at filter at <console>:33

Now, let's list the 404 response code records. Using the RDD containing only log records with a 404 response code that you cached in the previous part, print out a list up to 10 distinct endpoints that generate 404 errors -  no endpoint should appear more than once in your list. You should receive the follwoing output as your result:
```
404 URLS: 
/SHUTTLE/COUNTDOWN
/shuttle/missions/sts-71/images/www.acm.uiuc.edu/rml/Gifs
/shuttle/technology/stsnewsrof/stsref-toc.html
/de/systems.html
/ksc.htnl
/~pccomp/graphics/sinsght.gif
/PERSONS/NASA-CM.
/shuttle/missions/sts-1/sts-1-mission.html
/history/apollo/sa-1/sa-1-patch-small.gif
/images/sts-63-Imax
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val badEndpoints = badRecords.
val badUniqueEndpoints = badEndpoints.<FILL IN>
val badUniqueEndpointsPick10 = badUniqueEndpoints.take(10)

println("404 URLS: ")
badUniqueEndpointsPick10.foreach(println)

Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top 10 endpoints that generate the most 404 errors. Remember, top endpoints should be in sorted order. The result would be as below:
```
Top ten 404 URLs: 
(/pub/winvn/readme.txt,633)
(/pub/winvn/release.txt,494)
(/shuttle/missions/STS-69/mission-STS-69.html,431)
(/images/nasa-logo.gif,319)
(/elv/DELTA/uncons.htm,178)
(/shuttle/missions/sts-68/ksc-upclose.gif,156)
(/history/apollo/sa-1/sa-1-patch-small.gif,146)
(/images/crawlerway-logo.gif,120)
(/://spacelink.msfc.nasa.gov,117)
(/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif,100)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val badEndpointsCountPairTuple = badRecords.map(<FILL IN>)
val badEndpointsSum = badEndpointsCountPairTuple.reduceByKey(<FILL IN>)

// sort the records (you should implement a new object by extending the Ordering class for this part)
object ErrOrdering extends Ordering[(String, Int)] {
  <FILL IN>
}

val badEndpointsTop10 = badEndpointsSum.takeOrdered(10)(ErrOrdering.reverse)

println("Top ten 404 URLs: ")
badEndpointsTop10.foreach(println)

Instead of looking at the endpoints that generated 404 errors, now let's look at the hosts that encountered 404 errors. Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top 10 hosts that generate the most 404 errors. Here is the result:
```
Top ten hosts that generated errors: 
(piweba3y.prodigy.com,39)
(maz3.maz.net,39)
(gate.barr.com,38)
(m38-370-9.mit.edu,37)
(ts8-1.westwood.ts.ucla.edu,37)
(nexus.mlckew.edu.au,37)
(204.62.245.32,33)
(spica.sci.isas.ac.jp,27)
(163.206.104.34,27)
(www-d4.proxy.aol.com,26)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val errHostsCountPairTuple = badRecords.map(<FILL IN>)
val errHostsSum = errHostsCountPairTuple.reduceByKey(<FILL IN>)
val errHostsTop10 = errHostsSum.takeOrdered(10)(<FILL IN>)

println("Top ten hosts that generated errors: ")
errHostsTop10.foreach(println)

Let's explore the 404 records temporally. Break down the 404 requests by day and get the daily counts sorted by day as a list. Since the log only covers a single month, you can ignore the month in your checks. Cache the `errDateSorted` at the end. The output should be as below:
```
404 errors by day: 
(1,243)
(3,303)
(4,346)
(5,234)
(6,372)
(7,532)
(8,381)
(9,279)
(10,314)
(11,263)
(12,195)
(13,216)
(14,287)
(15,326)
(16,258)
(17,269)
(18,255)
(19,207)
(20,312)
(21,305)
(22,288)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val errDateCountPairTuple = badRecords.map(<FILL IN>)
val errDateSum = errDateCountPairTuple.reduceByKey(<FILL IN>)
val errDateSorted = errDateSum
val errByDate = errDateSorted.takeOrdered(30)
errDateSorted.cache()

println("404 errors by day: ")
errByDate.foreach(println)

Using the RDD `errDateSorted` you cached before, what are the top five days for 404 response codes and the corresponding counts of 404 response codes?
```
Top five dates for 404 requests: (7,532) (8,381) (6,372) (4,346) (15,326) 
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

object DateOrdering extends Ordering[(Int, Int)] {
  <FILL IN>
}

val topErrDate = errDateSorted.takeOrdered(5)(DateOrdering.reverse)

print("Top five dates for 404 requests: ")
topErrDate.foreach(x => print(x + " "))

Using the RDD `badRecords` you cached before, and by hour of the day and in increasing order, create an RDD containing how many requests had a 404 return code for each hour of the day (midnight starts at 0). 
```
Top hours for 404 requests: 
(0,175)
(1,171)
(2,422)
(3,272)
(4,102)
(5,95)
(6,93)
(7,122)
(8,199)
(9,185)
(10,329)
(11,263)
(12,438)
(13,397)
(14,318)
(15,347)
(16,373)
(17,330)
(18,268)
(19,269)
(20,270)
(21,241)
(22,234)
(23,272)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val hourCountPairTuple = badRecords.map(<FILL IN>)
val hourRecordsSum = hourCountPairTuple.reduceByKey(<FILL IN>)
val hourRecordsSorted = hourRecordsSum.<FILL IN>
val errHourList = hourRecordsSorted.<FILL IN>

println("Top hours for 404 requests: ")
errHourList.foreach(println)